In [5]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import scipy.io as sio
import sys
from functools import partial

sys.path.append('./../')

import large_scale_UQ as luq

import optimusprimal as optpr

from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
plt.style.use('dark_background')
plt.rcParams["font.family"] = "serif"



In [6]:
!pwd

/Users/tliaudat/Documents/postdoc/github/large-scale-UQ/notebooks


First, we need to define some heuristics for the solver, these include:

      - tol: convergence criteria for the iterations
      - iter: maximum number of iterations
      - update_iter: iterations between logging iteration diagnostics
      - record_iters: whether to record the full diagnostic information



In [7]:
options = {"tol": 1e-5, "iter": 5000, "update_iter": 50, "record_iters": False}

In [12]:

img_path = '/Users/tliaudat/Documents/postdoc/github/large-scale-UQ/data/imgs/M31.fits'
img_data = fits.open(img_path, memmap=False)

# Loading the image and cast it to float
img = np.copy(img_data[0].data)[0,:,:].astype(np.float64)
# Flipping data
img = np.flipud(img)

# Aliases
x = img
ground_truth = img
 
# Load op from X Cai
op_mask = sio.loadmat('/Users/tliaudat/Documents/postdoc/github/large-scale-UQ/data/operators_masks/fourier_mask.mat')['Ma']

# Matlab's reshape works with 'F'-like ordering
mat_mask = np.reshape(np.sum(op_mask, axis=0), (256,256), order='F').astype(bool)




dim = img.shape[0]

# A mock radio imaging forward model with half of the Fourier coefficients masked
phi = luq.operators.MaskedFourier(dim, 0.5 , norm='ortho')
# Use X. Cai's Fourier mask
phi.mask = mat_mask

# Simulate mock noisy observations y
y = phi.dir_op(img)
ISNR = 25
sigma = np.sqrt(np.mean(np.abs(y)**2)) * 10**(-ISNR/20)

print(sigma)


0.00971288706369457


In [10]:
# Using the same noise as in X Cai
sigma = 0.0024

# Add noise
rng = np.random.default_rng(seed=0)
n = rng.normal(0, sigma, y.shape)

# Simulate mock noisy observations
y += n
# Define init img
x_init = np.abs(phi.adj_op(y))